### CAPSTONE PROJECT PART 1: EDA
The soluability of molecules is an important physical chemical property for drug discovery, design and development. 
This type of project is performed by pharmaceutical companies and academia.

I will be applying linear regression to predict the solubility of molecules.

The data for this project was found here:
https://pubs.acs.org/doi/10.1021/ci034243x#

In [26]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
# Read in the dataset and have a look
sol = pd.read_csv('data/delaney.csv')
sol

# *1st column is the compound ID which is also the name of the molecule.
# *2nd column is the actual value also known as the experimental value for solubility. 
# *3rd column is the predicted from the study done by John Delaney. 
# *4th column is the smiles notation, in bioinformatics smiles notation represents the-
# *acronym for simplified molecular input line entry system. 

# *It is essentially compressing the chemical structure information into a 1D form. 

,Compound ID,measured log(solubility:mol/L),ESOL predicted log(solubility:mol/L),SMILES
0,"1,1,1,2-Tetrachloroethane",-2.180,-2.794,ClCC(Cl)(Cl)Cl
1,"1,1,1-Trichloroethane",-2.000,-2.232,CC(Cl)(Cl)Cl
2,"1,1,2,2-Tetrachloroethane",-1.740,-2.549,ClC(Cl)C(Cl)Cl
3,"1,1,2-Trichloroethane",-1.480,-1.961,ClCC(Cl)Cl
4,"1,1,2-Trichlorotrifluoroethane",-3.040,-3.077,FC(F)(Cl)C(F)(Cl)Cl
...,...,...,...,...
1139,vamidothion,1.144,-1.446,CNC(=O)C(C)SCCSP(=O)(OC)(OC)
1140,Vinclozolin,-4.925,-4.377,CC1(OC(=O)N(C1=O)c2cc(Cl)cc(Cl)c2)C=C
1141,Warfarin,-3.893,-3.913,CC(=O)CC(c1ccccc1)c3c(O)c2ccccc2oc3=O
1142,Xipamide,-3.790,-3.642,Cc1cccc(C)c1NC(=O)c2cc(c(Cl)cc2O)S(N)(=O)=O


### Exploring the SMILE data
* Chemical structures are encoded by a string of text known as `SMILES`

In [3]:
sol.SMILES

0                                    ClCC(Cl)(Cl)Cl
1                                      CC(Cl)(Cl)Cl
2                                    ClC(Cl)C(Cl)Cl
3                                        ClCC(Cl)Cl
4                               FC(F)(Cl)C(F)(Cl)Cl
                           ...                     
1139                   CNC(=O)C(C)SCCSP(=O)(OC)(OC)
1140          CC1(OC(=O)N(C1=O)c2cc(Cl)cc(Cl)c2)C=C
1141         CC(=O)CC(c1ccccc1)c3c(O)c2ccccc2oc3=O 
1142    Cc1cccc(C)c1NC(=O)c2cc(c(Cl)cc2O)S(N)(=O)=O
1143                         CNC(=O)Oc1cc(C)cc(C)c1
Name: SMILES, Length: 1144, dtype: object

### Convert a molecule from the SMILES string to an rdkit object.
In order to allow the rdkit to comprehend the information of the chemical structure, I will need to read it into rdkit by converting the smiles notation into the rdkit object

In [5]:
# This will convert the smiles into the object
Chem.MolFromSmiles(sol.SMILES[0])

### Working with the rdkit object

In [6]:
# I will now assign this rdkit object to the variable 'm' and then I will use a function to-
# see how many number of atoms it has. 
m = Chem.MolFromSmiles(sol.SMILES[0])

In [7]:
# This compound has 6 atoms. This essentially computes the number of atoms the molecule has. 
m.GetNumAtoms()

6

### Calculate Molecular Descriptors in rdkit

In [8]:
# TODO: Convert list of molecules into rdkit objects

# This will create an empty list and for each of the smiles notation in the sol.SMILES and iterate through each one-
# converting them into rdkit objects.
mol_list = [Chem.MolFromSmiles(element) for element in sol.SMILES]

In [9]:
len(mol_list)

1144

### Calculate molecular descriptors
To predict LogS(log of the aqueous solubility) the study by Delaney makes use of 4 molecular descriptors

    1. cLogP (Octanol-water partition coefficient) - the ratio between solubility of a molecule in optinal and water
    2. MW (Molecular weight)
    3. RB (Number of rotatable bonds)
    4. AP (Aromatic proportion = number of aromatic atoms / total number of heavy atoms)

Unfortunately, rdkit readily computes the first 3. As for the AP descriptor, I will calculate this by manually computing the ratio of the number of aromatic atoms to the total number of heavy atoms which rdkit can compute. 

In [12]:
# TODO: descriptors to be calculated are LogP, MW, and RB
# Inspired by: https://codeocean.com/explore/capsule?query=tag:data-curation

def generate(smiles, verbose=False):
    # This will create an empty list and for each of the smiles notation in the sol.SMILES and iterate through each one-
    # converting them into rdkit objects.
    moldata = []
    for elem in smiles:
        mol = Chem.MolFromSmiles(elem)
        moldata.append(mol)
    
    baseData = np.arange(1,1)
    i = 0
    # computes the 3 descriptors and then converts them into the numpy array.
    for mol in moldata:
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_NumRotatableBonds = Descriptors.NumRotatableBonds(mol)
        
        row = np.array([desc_MolLogP,
                        desc_MolWt,
                        desc_NumRotatableBonds])
        
        if(i==0):
            baseData = row
        else:
            baseData = np.vstack([baseData, row])
        i = i+1
    
    #dumps the above code into a dataframe 
    columnNames = ["MolLogP", "MolWt", "NumRotatableBonds"]
    descriptors = pd.DataFrame(data=baseData, columns=columnNames)
    
    return descriptors
        

In [13]:
# TODO: Generate the descriptor and put it into a new dataframe 
df = generate(sol.SMILES)
df

,MolLogP,MolWt,NumRotatableBonds
0,2.59540,167.850,0.0
1,2.37650,133.405,0.0
2,2.59380,167.850,1.0
3,2.02890,133.405,1.0
4,2.91890,187.375,1.0
...,...,...,...
1139,1.98820,287.343,8.0
1140,3.42130,286.114,2.0
1141,3.60960,308.333,4.0
1142,2.56214,354.815,3.0


### Calculating the 4th descriptor
* Here I will create a custom function to calculate the `number of aromatic atoms`.
* With this descriptor I can use it to subsequently calculate the AP

In [14]:
# TODO: computing for a single molecule 
m = Chem.MolFromSmiles('CC1(OC(=O)N(C1=O)c2cc(Cl)cc(Cl)c2)C=C')

In [15]:
# For the molecular object above, I am going to retrieve all of the atoms from the molecule-
# and then for each atom I'm going to determine whether it is aromatic or not.

# This will return a Boolean where aromatic is true and non-aromatic is false
aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
aromatic_atoms

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 False]

In [18]:
# TODO: create function that returns the above count
def AromaticAtoms(m):
    aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
    aa_count = []
    for i in aromatic_atoms:
        if i == True:
            aa_count.append(1)
    
    sum_aa_count = sum(aa_count)
    return sum_aa_count

In [19]:
AromaticAtoms(m)

6

In [20]:
# TODO: computing the molecules in the entire dataset 
desc_AromaticAtoms = [AromaticAtoms(element) for element in mol_list]
desc_AromaticAtoms

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 0,
 6,
 0,
 0,
 0,
 0,
 6,
 6,
 0,
 6,
 6,
 6,
 6,
 6,
 0,
 6,
 6,
 0,
 0,
 6,
 10,
 6,
 6,
 0,
 6,
 6,
 6,
 6,
 10,
 6,
 0,
 10,
 0,
 14,
 0,
 0,
 14,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 12,
 10,
 14,
 6,
 10,
 10,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 0,
 12,
 12,
 12,
 12,
 0,
 0,
 12,
 0,
 0,
 0,
 0,
 0,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 6,
 6,
 12,
 12,
 6,
 0,
 6,
 12,
 6,
 6,
 6,
 6,
 0,
 0,
 10,
 0,
 6,
 12,
 12,
 6,
 12,
 6,
 6,
 6,
 6,
 0,
 0,
 0,
 0,
 6,
 6,
 6,
 12,
 12,
 6,
 10,
 6,
 6,
 6,
 12,
 10,
 14,
 10,
 10,
 0,
 6,
 0,
 0,
 0,
 0,
 6,
 12,
 0,
 10,
 6,
 0,
 6,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 10,
 6,
 0,
 0,
 0,
 0,
 10,
 6,
 0,
 6,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 14,
 0,

### Number of heavy atoms
* I will use an existing function to calculate the number of heavy atoms

In [21]:
# TODO: computing for molecules in the entire dataset
desc_HeavyAtomCount = [Descriptors.HeavyAtomCount(element) for element in mol_list]
desc_HeavyAtomCount

[6,
 5,
 6,
 5,
 8,
 4,
 4,
 8,
 10,
 10,
 10,
 9,
 9,
 10,
 10,
 10,
 9,
 9,
 9,
 8,
 8,
 4,
 8,
 4,
 5,
 8,
 8,
 10,
 12,
 4,
 9,
 9,
 9,
 15,
 8,
 4,
 8,
 8,
 5,
 8,
 8,
 12,
 12,
 8,
 6,
 8,
 8,
 10,
 8,
 12,
 12,
 5,
 12,
 6,
 14,
 11,
 22,
 15,
 5,
 5,
 8,
 7,
 11,
 9,
 6,
 4,
 5,
 4,
 4,
 4,
 5,
 5,
 8,
 7,
 11,
 6,
 4,
 11,
 10,
 13,
 12,
 8,
 7,
 7,
 17,
 7,
 6,
 7,
 6,
 5,
 8,
 11,
 4,
 7,
 14,
 11,
 15,
 9,
 11,
 11,
 13,
 6,
 10,
 9,
 9,
 19,
 9,
 8,
 8,
 16,
 6,
 5,
 5,
 9,
 4,
 15,
 22,
 20,
 18,
 20,
 18,
 16,
 19,
 19,
 18,
 17,
 17,
 18,
 16,
 7,
 18,
 18,
 16,
 17,
 8,
 9,
 16,
 7,
 6,
 7,
 8,
 6,
 14,
 18,
 19,
 18,
 17,
 17,
 16,
 11,
 11,
 15,
 15,
 10,
 8,
 11,
 15,
 10,
 10,
 11,
 9,
 6,
 6,
 12,
 7,
 8,
 15,
 15,
 10,
 15,
 10,
 10,
 16,
 9,
 8,
 8,
 8,
 7,
 9,
 8,
 13,
 14,
 14,
 9,
 12,
 9,
 8,
 13,
 14,
 12,
 15,
 11,
 11,
 4,
 8,
 5,
 5,
 8,
 6,
 9,
 13,
 5,
 11,
 8,
 4,
 8,
 6,
 11,
 8,
 7,
 9,
 9,
 7,
 9,
 12,
 9,
 8,
 8,
 7,
 7,
 11,
 7,
 4,
 10,
 12,
 5,

In [22]:
# TODO: Computing the aromatic proportion (AP) descriptor
# Now I will take the two variables, aromatic atoms and heavy atoms, and I will divide them.

desc_AP = [AromaticAtoms(element)/Descriptors.HeavyAtomCount(element) for element in mol_list]
desc_AP

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6,
 0.6,
 0.6,
 0.6666666666666666,
 0.6666666666666666,
 0.6,
 0.6,
 0.6,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.75,
 0.75,
 0.0,
 0.75,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6,
 0.5,
 0.0,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.4,
 0.75,
 0.0,
 0.75,
 0.75,
 0.0,
 0.0,
 0.75,
 0.8333333333333334,
 0.5,
 0.75,
 0.0,
 0.75,
 0.75,
 0.6,
 0.75,
 0.8333333333333334,
 0.5,
 0.0,
 0.8333333333333334,
 0.0,
 1.0,
 0.0,
 0.0,
 0.9333333333333333,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9090909090909091,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9090909090909091,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8333333333333334,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9090909090909091,
 0.0,
 0.0,
 0.8571428571428571,
 0.9090909090909091,
 0.9333333333333333,
 0.6666666666666666,
 0.9090909090909091,
 0.9090909090909091,
 0.7692307692307693,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0

In [23]:
# Now I will turn the above output into a dataframe
df_desc_AP = pd.DataFrame(desc_AP, columns=['AromaticProportion'])
df_desc_AP

,AromaticProportion
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
1139,0.000000
1140,0.333333
1141,0.695652
1142,0.521739


### X matrix (Combining all computed descriptors into 1 dataframe)
* Here I will combine the two dataframes together. 
* The `df` which contains the three molecular descriptors.
* The `df_desc_AP` which is the 4th molar descriptor.

In [24]:
# TODO: combine the 2 dataframes to produce the X matrix
# I will use the X matrix in scikit learn for the model building. 
X = pd.concat([df, df_desc_AP], axis=1)
X

,MolLogP,MolWt,NumRotatableBonds,AromaticProportion
0,2.59540,167.850,0.0,0.000000
1,2.37650,133.405,0.0,0.000000
2,2.59380,167.850,1.0,0.000000
3,2.02890,133.405,1.0,0.000000
4,2.91890,187.375,1.0,0.000000
...,...,...,...,...
1139,1.98820,287.343,8.0,0.000000
1140,3.42130,286.114,2.0,0.333333
1141,3.60960,308.333,4.0,0.695652
1142,2.56214,354.815,3.0,0.521739


### Y matrix
* This is for the solubility of the molecule (LogS)

In [25]:
y = sol.iloc[:,1]
y

0      -2.180
1      -2.000
2      -1.740
3      -1.480
4      -3.040
        ...  
1139    1.144
1140   -4.925
1141   -3.893
1142   -3.790
1143   -2.581
Name: measured log(solubility:mol/L), Length: 1144, dtype: float64

### Data Split
* I now have the data matrices of the x and y ready for splitting
* I will set the ratio to 80/20

In [27]:
# set test size to 20% because I want 80% to be the train set.  
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

### Linear Regression Model

In [28]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [29]:
# Now that I have the trained model I can apply it to make the prediction on X_train and X_test
# TODO: Predict the X_train

y_pred_train = model.predict(X_train)

In [30]:
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('Mean Squared Error (MSE): %.2f' % mean_squared_error(y_train, y_pred_train))
print('Coefficient of Determination (R^2): %.2f' % r2_score(y_train, y_pred_train))

Coefficients: [-0.76663073 -0.00620568 -0.00382642 -0.41512124]
Intercept: 0.27234841745350513
Mean Squared Error (MSE): 0.99
Coefficient of Determination (R^2): 0.78


In [31]:
# Apply the model on the X_test
# TODO: Predict the X_test

y_pred_test = model.predict(X_test)

In [32]:
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('Mean Squared Error (MSE): %.2f' % mean_squared_error(y_test, y_pred_test))
print('Coefficient of Determination (R^2): %.2f' % r2_score(y_test, y_pred_test))

Coefficients: [-0.76663073 -0.00620568 -0.00382642 -0.41512124]
Intercept: 0.27234841745350513
Mean Squared Error (MSE): 1.09
Coefficient of Determination (R^2): 0.73
